<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e2c1cb643cd9bdb476ad54fe9afdcbe0ff860f0f4cf4f7b81f2df37f9321ecfd
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4


    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-16 13:22:35
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: 2.10 K (0.01%)
Current PnL: -28.78 L (-18.35%)
CY Booked + Current PnL: -14.49 L (-9.24%)
-------------------
Total profit:  1.91 L
Total loss:  -30.68 L
-------------------
Total Booked + Current PnL: 12.49 L (9.62%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.19%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.24 L (67.17%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.18%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.73,16.07,4.59,21.40,14141.0,42658.0,308084.0,7.72,63.0,X-LC,6.60,14.0,3.02,2.20,23.42,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.86,5.37,13.15,19.23,20849.0,8084.0,158545.0,24.27,66.0,L-LC,9.22,182.0,0.39,1.13,27.79,XY25,ATH,MINING
77,TTKPRESTIG,770.00,1.01,-22.27,28.76,0.09,22530.0,-22439.0,78338.0,81.86,37.0,M-SC,3.86,253.0,-1.00,0.56,2.92,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.32,-14.48,16.95,0.02,25304.0,-25275.0,149284.0,127.68,52.0,M-SC,9.08,240.0,-1.00,1.06,29.85,OX40N,NTT,PAINTS
49,MASFIN,398.61,0.00,-4.73,28.11,22.05,26239.0,-4635.0,93345.0,-17.80,45.0,H-SC,5.53,168.0,-0.18,0.67,35.68,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,8.39,-1.51,109.89,106.72,133490.0,-1860.0,121476.0,-51.20,59.0,H-SC,8.08,138.0,-0.01,0.87,21.52,AR,ATH,MISC
41,INFY,1972.00,5.31,14.69,17.05,34.24,61186.0,45974.0,358862.0,-10.62,64.0,X-LC,7.69,6.0,0.75,2.56,24.19,X40,NTT,IT
55,RAJESHEXPO,518.00,5.00,-66.52,198.87,0.07,91341.0,-91247.0,45930.0,1583.08,42.0,L-SC,0.75,269.0,-1.00,0.33,14.23,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,3.83,-21.02,90.91,50.79,100993.0,-29562.0,111091.0,1.28,54.0,H-SC,10.20,171.0,-0.29,0.79,29.35,XR,ATH,IT
18,COFFEEDAY,80.00,3.00,-41.97,133.10,35.27,87705.0,-47655.0,65894.0,-55.73,46.0,L-SC,4.71,268.0,-0.54,0.47,61.28,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-3.63,-47.35,317.28,119.69,452283.0,-128211.0,142550.0,-47.02,25.0,H-SC,12.57,150.0,-0.28,1.02,12.61,AR,ATH,IT
22,DIXON,18931.72,-2.86,-25.22,75.54,31.26,130352.0,-58208.0,172560.0,-65.07,22.0,X-MC,21.55,56.0,-0.45,1.23,0.00,X40N,ATH,IT
80,VALIANTORG,838.00,-2.86,-48.21,236.28,74.15,200195.0,-78877.0,84728.0,-252.21,34.0,H-SC,22.09,145.0,-0.39,0.60,8.61,XR,NTT,CHEMICALS
4,ALKYLAMINE,4546.37,-2.56,-31.83,184.04,93.63,126667.0,-32137.0,68826.0,-27.06,49.0,H-SC,16.54,143.0,-0.25,0.49,4.05,SR,ATH,CHEMICALS
45,JIOFIN,387.00,-1.99,-9.52,37.58,24.48,90913.0,-25447.0,241918.0,-11.00,35.0,X-LC,1.69,37.0,-0.28,1.72,40.22,XY24,BTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.62,0.76,19.83,20.74,40594.0,1540.0,204710.0,33.49,57.0,M-MC,5.18,189.0,0.04,1.46,11.45,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.59,-1.02,110.05,107.91,160827.0,-1504.0,146140.0,-19.34,56.0,M-SC,9.95,220.0,-0.01,1.04,8.97,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.32,-14.48,16.95,0.02,25304.0,-25275.0,149284.0,127.68,52.0,M-SC,9.08,240.0,-1.00,1.06,29.85,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,1.84,-19.20,63.40,32.03,67536.0,-25311.0,106524.0,-12.51,57.0,H-SC,6.57,157.0,-0.37,0.76,19.32,OX40N,NTT,IT
77,TTKPRESTIG,770.00,1.01,-22.27,28.76,0.09,22530.0,-22439.0,78338.0,81.86,37.0,M-SC,3.86,253.0,-1.00,0.56,2.92,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.88,-22.53,46.48,13.47,97096.0,-60768.0,208899.0,-68.12,47.0,H-SC,3.96,159.0,-0.63,1.49,6.22,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.62,0.76,19.83,20.74,40594.0,1540.0,204710.0,33.49,57.0,M-MC,5.18,189.0,0.04,1.46,11.45,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.86,5.37,13.15,19.23,20849.0,8084.0,158545.0,24.27,66.0,L-LC,9.22,182.0,0.39,1.13,27.79,XY25,ATH,MINING
84,WIPRO,420.00,2.96,9.88,56.77,72.27,105700.0,16742.0,186190.0,-5.63,57.0,M-LC,6.35,101.0,0.16,1.33,16.67,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,2.96,9.88,56.77,72.27,105700.0,16742.0,186190.0,-5.63,57.0,M-LC,6.35,101.0,0.16,1.33,16.67,XR,NTT,IT
38,INDIAMART,4810.62,8.39,-1.51,109.89,106.72,133490.0,-1860.0,121476.0,-51.20,59.0,H-SC,8.08,138.0,-0.01,0.87,21.52,AR,ATH,MISC
25,FINCABLES,1641.55,0.59,-1.02,110.05,107.91,160827.0,-1504.0,146140.0,-19.34,56.0,M-SC,9.95,220.0,-0.01,1.04,8.97,OX40N,ATH,CABLES
37,IEX,219.00,0.45,-4.99,56.43,48.63,107206.0,-9974.0,189980.0,-36.71,49.0,H-SC,13.21,137.0,-0.09,1.35,6.05,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.79,-7.54,46.70,35.63,90487.0,-15812.0,193762.0,-20.73,31.0,H-MC,3.78,119.0,-0.17,1.38,7.87,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.36,-19.07,36.89,10.78,71747.0,-45830.0,194488.0,-51.78,12.0,X-LC,9.34,1.0,-0.64,1.39,0.00,X40,NTT,FMCG
7,AWL,485.00,-1.85,-30.68,126.76,57.19,286568.0,-100056.0,226071.0,-67.20,18.0,X-MC,11.01,58.0,-0.35,1.61,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-2.86,-25.22,75.54,31.26,130352.0,-58208.0,172560.0,-65.07,22.0,X-MC,21.55,56.0,-0.45,1.23,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,0.06,-41.54,131.28,35.20,98748.0,-53451.0,75219.0,-1.37,26.0,X-SC,17.59,93.0,-0.54,0.54,0.06,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.56,-8.08,45.98,34.19,90246.0,-17248.0,196272.0,-35.33,28.0,X-MC,4.62,60.0,-0.19,1.40,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JIOFIN,387.00,-1.99,-9.52,37.58,24.48,90913.0,-25447.0,241918.0,-11.00,35.0,X-LC,1.69,37.0,-0.28,1.72,40.22,XY24,BTT,FINANCE
73,TCS,4389.97,0.44,-11.83,36.90,20.71,123060.0,-44740.0,333497.0,-23.95,47.0,X-LC,1.88,3.0,-0.36,2.38,11.43,X40,ATH,IT
78,UNITDSPR,1644.00,0.76,-1.11,22.12,20.77,51218.0,-2598.0,231546.0,-9.72,39.0,X-MC,1.97,70.0,-0.05,1.65,5.73,X40N,NTT,BREWERIES
20,DABUR,735.00,0.17,0.61,42.83,43.70,106896.0,1513.0,249581.0,-7.59,55.0,X-MC,2.14,73.0,0.01,1.78,15.75,XY24,BTT,FMCG
65,SIEMENS,4671.50,-1.49,-21.37,59.63,25.51,87254.0,-39770.0,146325.0,-10.26,34.0,H-LC,2.49,50.0,-0.46,1.04,7.83,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.85,-30.68,126.76,57.19,286568.0,-100056.0,226071.0,-67.20,18.0,X-MC,11.01,58.0,-0.35,1.61,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.72,-20.49,95.54,55.47,184810.0,-49849.0,193437.0,-29.54,33.0,X-MC,15.90,63.0,-0.27,1.38,0.0,X40N,ATH,FINANCE
9,BAJAJHLDNG,14451.52,-0.98,-4.90,35.87,29.22,64857.0,-9313.0,180812.0,-7.33,34.0,X-LC,12.00,29.0,-0.14,1.29,0.0,X40,ATH,FINANCE
22,DIXON,18931.72,-2.86,-25.22,75.54,31.26,130352.0,-58208.0,172560.0,-65.07,22.0,X-MC,21.55,56.0,-0.45,1.23,0.0,X40N,ATH,IT
58,RELAXO,1176.00,-0.56,-49.37,200.84,52.31,147595.0,-71671.0,73489.0,-46.08,33.0,X-SC,8.03,92.0,-0.49,0.52,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.80,-30.07,104.26,42.84,47397.0,-19546.0,45460.0,-54.27,43.0,X-SC,24.88,83.0,-0.41,0.32,2.85,XY24,NTT,MISC
58,RELAXO,1176.00,-0.56,-49.37,200.84,52.31,147595.0,-71671.0,73489.0,-46.08,33.0,X-SC,8.03,92.0,-0.49,0.52,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.06,-41.54,131.28,35.20,98748.0,-53451.0,75219.0,-1.37,26.0,X-SC,17.59,93.0,-0.54,0.54,0.06,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.08,-3.98,29.22,24.08,35619.0,-5050.0,121900.0,-11.75,42.0,X-SC,5.88,91.0,-0.14,0.87,14.86,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.25,-17.81,74.38,43.32,99565.0,-29012.0,133860.0,-29.42,45.0,X-SC,8.20,90.0,-0.29,0.95,2.78,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.36,-19.07,36.89,10.78,71747.0,-45830.0,194488.0,-51.78,12.0,X-LC,9.34,1.0,-0.64,1.39,0.00,X40,NTT,FMCG
75,TMPV,600.00,2.12,-26.79,67.97,22.98,112412.0,-60514.0,165384.0,-24.30,47.0,X-LC,3.60,2.0,-0.54,1.18,4.02,XY24,NTT,AUTO
73,TCS,4389.97,0.44,-11.83,36.90,20.71,123060.0,-44740.0,333497.0,-23.95,47.0,X-LC,1.88,3.0,-0.36,2.38,11.43,X40,ATH,IT
81,VBL,671.64,-0.10,1.38,33.82,35.67,108296.0,4370.0,320212.0,-10.61,60.0,X-LC,2.78,5.0,0.04,2.28,15.43,X40N,ATH,FMCG
41,INFY,1972.00,5.31,14.69,17.05,34.24,61186.0,45974.0,358862.0,-10.62,64.0,X-LC,7.69,6.0,0.75,2.56,24.19,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.91,-29.64,88.87,32.89,48278.0,-22882.0,54324.0,-770.56,74.0,L-MC,9.21,259.0,-0.47,0.39,49.60,XR,NTT,BANKS
13,BSOFT,831.70,3.83,-21.02,90.91,50.79,100993.0,-29562.0,111091.0,1.28,54.0,H-SC,10.20,171.0,-0.29,0.79,29.35,XR,ATH,IT
69,SURYODAY,216.00,1.12,-19.07,52.03,23.03,75403.0,-34149.0,144922.0,57.89,52.0,H-SC,4.45,165.0,-0.45,1.03,43.79,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.32,-14.48,16.95,0.02,25304.0,-25275.0,149284.0,127.68,52.0,M-SC,9.08,240.0,-1.00,1.06,29.85,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.86,5.37,13.15,19.23,20849.0,8084.0,158545.0,24.27,66.0,L-LC,9.22,182.0,0.39,1.13,27.79,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.29,-28.12,57.07,12.90,89635.0,-61435.0,157061.0,131.97,60.0,H-SC,8.97,173.0,-0.69,1.12,30.59,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,0.91,-29.64,88.87,32.89,48278.0,-22882.0,54324.0,-770.56,74.0,L-MC,9.21,259.0,-0.47,0.39,49.60,XR,NTT,BANKS
72,TATAELXSI,9161.00,1.84,-19.20,63.40,32.03,67536.0,-25311.0,106524.0,-12.51,57.0,H-SC,6.57,157.0,-0.37,0.76,19.32,OX40N,NTT,IT
17,COALINDIA,484.83,-0.86,5.37,13.15,19.23,20849.0,8084.0,158545.0,24.27,66.0,L-LC,9.22,182.0,0.39,1.13,27.79,XY25,ATH,MINING
57,RECLTD,446.00,0.62,0.76,19.83,20.74,40594.0,1540.0,204710.0,33.49,57.0,M-MC,5.18,189.0,0.04,1.46,11.45,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.50
1,25,44.58
2,50,76.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.74
MC    30.46
LC    26.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.28
X40      24.04
X40N     12.73
XY25     10.83
XR        8.99
AR        8.41
OX40N     7.74
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.10
X-LC    23.27
H-SC    22.86
M-SC    11.22
X-SC     7.86
H-MC     4.51
M-MC     1.46
M-LC     1.33
L-LC     1.13
H-LC     1.04
L-SC     0.80
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.16,-9.43,45.33
IT,12.91,-18.03,77.56
FINANCE,11.10,-14.51,62.57
MISC,7.11,-30.12,83.35
PAINTS,5.81,-13.19,30.01
ELECTRICAL,5.79,-15.39,57.66
INSURANCE,4.70,-2.77,37.94
PHARMA,3.92,-6.98,40.66
AUTO,2.82,-32.70,78.70


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3281211.0,21
AR,1392046.0,10
XR,1229659.0,12
X40,1149489.0,15
X40N,954510.0,9
OX40N,706638.0,10
XY25,423448.0,7
SR,286624.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3584727.0,24
X-MC,1652277.0,16
M-SC,1485088.0,15
X-LC,958877.0,13
X-SC,815552.0,8
H-MC,445383.0,3
L-SC,179046.0,2
M-LC,105700.0,1
H-LC,87254.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260171.0      6
           AR         935508.0      5
M-SC       XY24       852021.0      6
H-SC       XR         797389.0      7
X-MC       X40        538106.0      7
           XY24       456694.0      3
           X40N       429319.0      4
X-LC       X40        415947.0      6
X-SC       X40N       319433.0      3
M-SC       OX40N      310262.0      5
H-SC       OX40N      305035.0      4
X-SC       XY24       300683.0      3
H-SC       SR         286624.0      2
H-MC       AR         237066.0      2
X-MC       XY25       228158.0      2
H-MC       XY24       208317.0      1
X-LC       X40N       205758.0      2
           XY24       203325.0      2
X-SC       X40        195436.0      2
M-SC       XR         190587.0      2
X-LC       XY25       133847.0      3
M-SC       AR         132218.0      2
M-LC       XR         105700.0      1
L-SC       OX40N       91341.0      1
           XR          87705.0      1
H-LC       AR          87254.0      1
L-MC       XR          48278.0      1
M-MC       XY25        40594.0      1
L-LC       XY25        20849.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
